In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> wordnet



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [ ]:
import nltk
import random
import json
import pickle
import numpy as np

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
lemmatizer= WordNetLemmatizer()

intents=json.loads(open("drive/MyDrive/Medicalchatbot/intents.json").read())

In [ ]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'Is anyone there?',
    'Hi there',
    'Hello',
    'Hey there',
    'Howdy',
    'Hola',
    'Bonjour',
    'Konnichiwa',
    'Guten tag',
    'Ola'],
   'responses': ['Hello there. Tell me how are you feeling today?',
    'Hi there. What brings you here today?',
    'Hi there. How are you feeling today?',
    'Great to see you. How do you feel currently?',
    "Hello there. Glad to see you're back. What's going on in your world right now?"]},
  {'tag': 'morning',
   'patterns': ['Good morning'],
   'responses': ["Good morning. I hope you had a good night's sleep. How are you feeling today? "]},
  {'tag': 'afternoon',
   'patterns': ['Good afternoon'],
   'responses': ['Good afternoon. How is your day going?']},
  {'tag': 'evening',
   'patterns': ['Good evening'],
   'responses': ['Good evening. How has your day been?']},
  {'tag': 'night',
   'patterns': ['Good night'],
   'responses': ['Good night. Get some prop

In [ ]:
words= []
classes= []
documents=[]
ignore_letter=['?', '!', '.', ',']

In [ ]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))

    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [ ]:
print(documents)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hi', 'there'], 'greeting'), (['Hello'], 'greeting'), (['Hey', 'there'], 'greeting'), (['Howdy'], 'greeting'), (['Hola'], 'greeting'), (['Bonjour'], 'greeting'), (['Konnichiwa'], 'greeting'), (['Guten', 'tag'], 'greeting'), (['Ola'], 'greeting'), (['Good', 'morning'], 'morning'), (['Good', 'afternoon'], 'afternoon'), (['Good', 'evening'], 'evening'), (['Good', 'night'], 'night'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Au', 'revoir'], 'goodbye'), (['Sayonara'], 'goodbye'), (['ok', 'bye'], 'goodbye'), (['Bye', 'then'], 'goodbye'), (['Fare', 'thee', 'well'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thanks', 'for', 'the', 'help'], 'thanks'), (['Than', 'you', 'very', 'much'], 'thanks'), ([], 'no-response'), (['nothing', 'much'], 'neutral-response'), (['Who', 'are', 'you', '?'], 'abou

In [ ]:
words= [lemmatizer.lemmatize(word) for word in words if word not in ignore_letter]
words=sorted(set(words))

In [ ]:
print(words)

["'ll", "'m", "'re", "'s", "'ve", 'All', 'Am', 'Are', 'Au', 'Bonjour', 'Bye', 'Can', 'Could', 'Define', 'Depression', 'Do', 'Fare', 'Fine', 'Good', 'Goodbye', 'Guten', 'Health', 'Hello', 'Hey', 'Hi', 'Hola', 'How', 'Howdy', 'I', 'If', 'Is', 'Just', 'K', 'Konnichiwa', 'Let', 'Mental', 'My', 'No', 'Nobody', 'Nothing', 'Oh', 'Ola', 'Probably', 'Sayonara', 'See', 'Someone', 'Support', 'Tell', 'Than', 'Thank', 'Thanks', 'That', 'What', 'Whatever', 'Where', 'Who', 'Why', 'Wrong', 'Yeah', 'You', 'a', 'about', 'absolutely', 'advice', 'affect', 'afternoon', 'again', 'all', 'alot', 'already', 'am', 'and', 'another', 'answer', 'anxiety', 'anxious', 'any', 'anymore', 'anyone', 'anything', 'appears', 'approaching', 'are', 'ask', 'available', 'away', 'awful', 'be', 'because', 'become', 'before', 'better', 'between', 'boyfriend', 'break', 'bring', 'brother', 'burned', 'by', 'bye', 'ca', 'call', 'can', 'cause', 'cheerful', 'child', 'commit', 'connection', 'continue', 'control', 'could', 'crazy', 'crea

In [ ]:
classes= sorted(set(classes))

In [ ]:
pickle.dump(words,open('drive/MyDrive/Medicalchatbot/words.pkl','wb'))
pickle.dump(classes,open('drive/MyDrive/Medicalchatbot/classes.pkl','wb'))

In [ ]:
training=[]
output_empty= [0] *len(classes)

for document in documents:
  bag=[]
  word_patterns=document[0]
  word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row=list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

In [ ]:
random.shuffle(training)
training=np.array(training)

train_x= list(training[:,0])
train_y= list(training[:,1])

<ipython-input-21-d0114324d694>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [ ]:
from re import VERBOSE
model=Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd= SGD(lr=0.01,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5,verbose=1)
model.save('medical_chatbot_model.hs',hist)
print('Done')

Epoch 1/200
47/47 [==============================] - 1s 3ms/step - loss: 4.3642 - accuracy: 0.0517
Epoch 2/200
47/47 [==============================] - 0s 3ms/step - loss: 4.2535 - accuracy: 0.0474
Epoch 3/200
47/47 [==============================] - 0s 3ms/step - loss: 4.1896 - accuracy: 0.0647
Epoch 4/200
47/47 [==============================] - 0s 3ms/step - loss: 4.0589 - accuracy: 0.0776
Epoch 5/200
47/47 [==============================] - 0s 4ms/step - loss: 3.9479 - accuracy: 0.0776
Epoch 6/200
47/47 [==============================] - 0s 4ms/step - loss: 3.8336 - accuracy: 0.0905
Epoch 7/200
47/47 [==============================] - 1s 13ms/step - loss: 3.6449 - accuracy: 0.1466
Epoch 8/200
47/47 [==============================] - 0s 3ms/step - loss: 3.4466 - accuracy: 0.1552
Epoch 9/200
47/47 [==============================] - 0s 3ms/step - loss: 3.3828 - accuracy: 0.1336
Epoch 10/200
47/47 [==============================] - 0s 4ms/step - loss: 3.2241 - accuracy: 0.1681
Epoch 11

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
lemmatizer=WordNetLemmatizer()
intents= json.loads(open("drive/MyDrive/Medicalchatbot/intents.json").read())

words= pickle.load(open("drive/MyDrive/Medicalchatbot/words.pkl",'rb'))
classes=pickle.load(open("drive/MyDrive/Medicalchatbot/classes.pkl",'rb'))
model=load_model('medical_chatbot_model.hs')

In [ ]:
def clean_up_sentence(sentence):
  sentence_words= nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [ ]:
def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0] *len(words)
  for w in sentence_words:
    for i , word in enumerate(words):
      if word==w:
        bag[i]=1
  return np.array(bag)

In [ ]:
def predict_class(sentence):
  bow=bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.25
  results=[[i,r] for i, r in enumerate(res) if r>ERROR_THRESHOLD]

  results.sort(key=lambda x:x[1], reverse=True)
  return_list=[]
  for r in results:
    return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
  return return_list


In [ ]:
def get_response(intents_list, intents_json):
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result



In [ ]:
print("GO! BOT IS RUNNING")

while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)

GO! BOT IS RUNNING
good morning
1/1 [==============================] - 0s 116ms/step
Good morning. I hope you had a good night's sleep. How are you feeling today? 
happy
1/1 [==============================] - 0s 24ms/step
Oh i see. That's great.
who made you
1/1 [==============================] - 0s 23ms/step
I was trained on a text dataset using Deep Learning & Natural Language Processing techniques
who is creator 
1/1 [==============================] - 0s 26ms/step
Therapy is a form of treatment that aims to help resolve mental or emotional issues. It is helpful for those with mental health conditions or even everyday life challenges.
what is mental health
1/1 [==============================] - 0s 26ms/step
Mental health is a state of well-being in which the individual realizes his or her own abilities, can cope with the normal stresses of life, can work productively and fruitfully, and is able to make a contribution to his or her community
how to improve mental health
1/1 [=========

KeyboardInterrupt: Interrupted by user